In [ ]:
import cv2 
import numpy as np


def ORB_detector(new_image, image_template):
    
    image1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    
    orb = cv2.ORB(1000, 1.2)
    
    (kp1, desc1) = orb.detectAndCompute(image1, None)
    
    (kp2, desc2) = orb.detectAndCompute(image_template, None)
    
    bf = cv2.BFMatcher(cv2.NORM_NAMING, crossCheck=True)
    
    matches = bf.match(desc1, desc2)
    
    matches = sorted(matches, key = lambda val: val.distance)
    
    return len(matches)



videostream = cv2.VideoCapture(0)

image_template = cv2.imread("template_live_detection.png", 0)

while True:
    
    #Get webcam images
    ret, frame = videostream.read()
    
    #Get height and width of webcam frame
    height, width =frame.shape[0:2]
    
    #define region of interest dimensions 
    top_left_x = width/3 #starts at a third of the screen width
    top_left_y = (height/2) + (height/4) # starts one sixth of the screen height
    bottom_right_x = (height/3) * 2 #starts at a 2 thirds of the height of the frame
    bottom_right_y = (height/2) - (height/4) 
    
    #cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (255, 0, 0), 3)
    
    #crop the window to obtain ROI coordinates
    cropped = frame[int(bottom_right_y): int(top_left_y) , int(top_left_x):int(bottom_right_x)]

    frame = cv2.flip(frame, 1)
    
    matches = ORB_detector(cropped, image_template)
    
    output_string = "Matches = {}".format(matches)
    cv2.putText(frame, output_string,(50, 450), cv2.FONT_HERSHEY_COMPLEX, 2, (250,0,250), 2)
    
    
    threshold = 500 #max number of matches 
    
    if matches > threshold:
        start_point = (top_left_x, top_left_y)
        end_point = (bottom_right_x, bottom_right_y)
        cv2.rectangle(frame, start_point, end_point, color = (0, 255, 0), thickness = 3)
        cv2.putText(frame,"Object Found",(50,50), cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0), 2)
        
    cv2.imshow("Object Detector using ORB", frame)
    
    if cv2.waitKey(1) == 13:
        break
        
    cap.release()
    cv2.destroyAllWindows()